In [2]:
import random, os, tqdm, time, json, re
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
from datetime import datetime

import sys
random.seed(618)
np.random.seed(907)

sys.path.append("../")
## mac环境
# new_base_path = os.path.join(
#     "/Users/minkexiu/Downloads/",
#     "/".join(
#         os.getcwd().split("/")[-1*(len(sys.path[-1].split("/")) - 1):]
#     ),
# )
## WSL环境
new_base_path = os.path.join(
    "/mnt/d/forCoding_data/",
    "/".join(
        os.getcwd().split("/")[-1*(len(sys.path[-1].split("/")) - 1):]
    ),
)
## wWindows环境
# sys.path.append("..\\..\\")
# new_base_path = os.path.join(
#     "D:\\forCoding_data\\",
#     "\\".join(
#         os.getcwd().split("\\")[-1*(len(sys.path[-1].split("\\")) - 1):]
#     ),
# )

print("storage dir:", new_base_path)
print("code dir:", os.getcwd())

storage dir: /mnt/d/forCoding_data/Tianchi_EcommerceKG
code dir: /mnt/d/forCoding_code/Tianchi_EcommerceKG


In [3]:
## 创建文件夹。
if not os.path.exists(new_base_path):
    os.makedirs(
        new_base_path
    )
if not os.path.exists(os.path.join(new_base_path, "preprocessedData")):
    os.makedirs(
        os.path.join(new_base_path, "preprocessedData")
    )
if not os.path.exists(os.path.join(new_base_path, "originalData")):
    os.makedirs(
        os.path.join(new_base_path, "originalData")
    )
if not os.path.exists(os.path.join(new_base_path, "trained_models")):
    os.makedirs(
        os.path.join(new_base_path, "trained_models")
    )

def create_originalData_path(filename_or_path):
    return os.path.join(new_base_path, "originalData", filename_or_path)
def create_preprocessedData_path(filename_or_path):
    return os.path.join(new_base_path, "preprocessedData", filename_or_path)
def create_trained_models_path(filename_or_path):
    return os.path.join(new_base_path, "trained_models", filename_or_path)

def millisec2datetime(timestamp):
    time_local = time.localtime(timestamp/1000)
    return time.strftime("%Y-%m-%d %H:%M:%S", time_local)
    
def run_finish():
    # 假设你的字体文件是 'myfont.ttf' 并且位于当前目录下  
    font = FontProperties(fname="/mnt/d/SimHei.ttf", size=24)  
    # 创建一个空白的图形  
    fig, ax = plt.subplots()  
    ax.imshow(
        
        plt.imread("/mnt/c/Users/Administrator/Pictures/Saved Pictures/ComfyUI_00044_.png")
    )
    # 在图形中添加文字  
    ax.text(
        ax.get_xlim()[1] * 0.5, 
        ax.get_ylim()[0] * 0.5, 
        f"程序于这个点跑完：\n{millisec2datetime(time.time()*1000)}", fontproperties=font, ha="center", va="center", color="red"
    )  
    # 设置图形的布局  
    # ax.set_xlim(0, 1)  
    # ax.set_ylim(0, 1)  
    ax.set_xticks([])  
    ax.set_yticks([])  
    ax.patch.set_color("blue")
    # 显示图形  
    plt.show()
        
tqdm.tqdm.pandas() ## 引入这个，就可以在apply的时候用progress_apply了。

import IPython
def kill_current_kernel():
    '''杀死当前的kernel释放内存空间。'''
    IPython.Application.instance().kernel.do_shutdown(True) 
    
def simply_show_data(df1):
    print(df1.shape)
    display(df1.head())
    
def wait_flag(saved_flag_path, time_interval_sec=10):
    print("waiting for", saved_flag_path)
    time_count = 0
    while True:
        if os.path.exists(saved_flag_path):
            break
        time.sleep(time_interval_sec)
        time_count+=time_interval_sec
        print(time_count, end=" ")
    print("finish!!")

class TimerContext:  
    def __enter__(self):  
        self.start_time = str(datetime.now())
        print("start time:", self.start_time)
        return self  
    def __exit__(self, exc_type, exc_val, exc_tb):  
        print("start time:", self.start_time)
        print("end time", str(datetime.now()))

In [35]:
train_data.num_nodes

14541

In [36]:
train_data.num_edge_types

237

In [37]:
train_data.

0

In [34]:
## 参考代码：https://github.com/pyg-team/pytorch_geometric/blob/master/examples/kge_fb15k_237.py 

import argparse
import os.path as osp

import torch
import torch.optim as optim

from torch_geometric.datasets import FB15k_237
from torch_geometric.nn import ComplEx, DistMult, RotatE, TransE

model_map = {
    'transe': TransE,
    'complex': ComplEx,
    'distmult': DistMult,
    'rotate': RotatE,
}

# parser = argparse.ArgumentParser()
# parser.add_argument('--model', choices=model_map.keys(), type=str.lower,
#                     required=True)
# args = parser.parse_args()

args_model = "transe"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
path = create_originalData_path("FB15k") # osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'FB15k')

train_data = FB15k_237(path, split='train')[0].to(device)
val_data = FB15k_237(path, split='val')[0].to(device)
test_data = FB15k_237(path, split='test')[0].to(device)

model_arg_map = {'rotate': {'margin': 9.0}}
model = model_map[args_model](
    num_nodes=train_data.num_nodes,
    num_relations=train_data.num_edge_types,
    hidden_channels=50,
    **model_arg_map.get(args_model, {}),
).to(device)

loader = model.loader(
    head_index=train_data.edge_index[0],
    rel_type=train_data.edge_type,
    tail_index=train_data.edge_index[1],
    batch_size=1000,
    shuffle=True,
)

optimizer_map = {
    'transe': optim.Adam(model.parameters(), lr=0.01),
    'complex': optim.Adagrad(model.parameters(), lr=0.001, weight_decay=1e-6),
    'distmult': optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-6),
    'rotate': optim.Adam(model.parameters(), lr=1e-3),
}
optimizer = optimizer_map[args_model]


def train():
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples


@torch.no_grad()
def test(data):
    model.eval()
    return model.test(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
        batch_size=20000,
        k=10,
    )


for epoch in range(1, 501):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch % 25 == 0:
        rank, mrr, hits = test(val_data)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}, '
              f'Val MRR: {mrr:.4f}, Val Hits@10: {hits:.4f}')

rank, mrr, hits_at_10 = test(test_data)
print(f'Test Mean Rank: {rank:.2f}, Test MRR: {mrr:.4f}, '
      f'Test Hits@10: {hits_at_10:.4f}')

Processing...
Done!


Epoch: 001, Loss: 0.7585
Epoch: 002, Loss: 0.5556
Epoch: 003, Loss: 0.4352
Epoch: 004, Loss: 0.3487
Epoch: 005, Loss: 0.2984
Epoch: 006, Loss: 0.2655
Epoch: 007, Loss: 0.2421
Epoch: 008, Loss: 0.2265
Epoch: 009, Loss: 0.2150
Epoch: 010, Loss: 0.2044
Epoch: 011, Loss: 0.1961
Epoch: 012, Loss: 0.1888
Epoch: 013, Loss: 0.1826
Epoch: 014, Loss: 0.1788
Epoch: 015, Loss: 0.1742
Epoch: 016, Loss: 0.1699
Epoch: 017, Loss: 0.1666
Epoch: 018, Loss: 0.1622
Epoch: 019, Loss: 0.1589
Epoch: 020, Loss: 0.1569
Epoch: 021, Loss: 0.1541
Epoch: 022, Loss: 0.1525
Epoch: 023, Loss: 0.1488
Epoch: 024, Loss: 0.1469
Epoch: 025, Loss: 0.1449


100%|███████████████████████████████████████████████████████████████████████████| 17535/17535 [00:06<00:00, 2775.94it/s]


Epoch: 025, Val Mean Rank: 404.47, Val MRR: 0.2236, Val Hits@10: 0.3670
Epoch: 026, Loss: 0.1436
Epoch: 027, Loss: 0.1422
Epoch: 028, Loss: 0.1404
Epoch: 029, Loss: 0.1384
Epoch: 030, Loss: 0.1371
Epoch: 031, Loss: 0.1357
Epoch: 032, Loss: 0.1342
Epoch: 033, Loss: 0.1327
Epoch: 034, Loss: 0.1314
Epoch: 035, Loss: 0.1307
Epoch: 036, Loss: 0.1294
Epoch: 037, Loss: 0.1280
Epoch: 038, Loss: 0.1270
Epoch: 039, Loss: 0.1266
Epoch: 040, Loss: 0.1251
Epoch: 041, Loss: 0.1250
Epoch: 042, Loss: 0.1247
Epoch: 043, Loss: 0.1221
Epoch: 044, Loss: 0.1219
Epoch: 045, Loss: 0.1217
Epoch: 046, Loss: 0.1204
Epoch: 047, Loss: 0.1198
Epoch: 048, Loss: 0.1189
Epoch: 049, Loss: 0.1179
Epoch: 050, Loss: 0.1173


100%|███████████████████████████████████████████████████████████████████████████| 17535/17535 [00:06<00:00, 2758.70it/s]


...

Epoch: 475, Val Mean Rank: 258.66, Val MRR: 0.2244, Val Hits@10: 0.3713
Epoch: 476, Loss: 0.0794
Epoch: 477, Loss: 0.0795
Epoch: 478, Loss: 0.0795
Epoch: 479, Loss: 0.0797
Epoch: 480, Loss: 0.0794
Epoch: 481, Loss: 0.0793
Epoch: 482, Loss: 0.0796
Epoch: 483, Loss: 0.0805
Epoch: 484, Loss: 0.0792
Epoch: 485, Loss: 0.0793
Epoch: 486, Loss: 0.0797
Epoch: 487, Loss: 0.0794
Epoch: 488, Loss: 0.0799
Epoch: 489, Loss: 0.0797
Epoch: 490, Loss: 0.0800
Epoch: 491, Loss: 0.0788
Epoch: 492, Loss: 0.0793
Epoch: 493, Loss: 0.0796
Epoch: 494, Loss: 0.0793
Epoch: 495, Loss: 0.0789
Epoch: 496, Loss: 0.0794
Epoch: 497, Loss: 0.0798
Epoch: 498, Loss: 0.0794
Epoch: 499, Loss: 0.0786
Epoch: 500, Loss: 0.0798


100%|███████████████████████████████████████████████████████████████████████████| 17535/17535 [00:06<00:00, 2765.14it/s]


Epoch: 500, Val Mean Rank: 258.11, Val MRR: 0.2254, Val Hits@10: 0.3715


100%|███████████████████████████████████████████████████████████████████████████| 20466/20466 [00:07<00:00, 2814.42it/s]

Test Mean Rank: 266.00, Test MRR: 0.2210, Test Hits@10: 0.3651


In [25]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import TransE
from torch.optim import Adam

# 定义节点和关系
# 假设 0: 人1, 1: 人2, 2: 公司1, 3: 公司2
# 边的定义 (source, target)
edge_index = torch.tensor([
    [0, 0, 1, 1],  # 投资人的索引
    [2, 3, 2, 3]   # 公司人的索引
], dtype=torch.long)

# 投资金额
edge_weight = torch.tensor([1000.0, 2000.0, 1500.0, 3000.0], dtype=torch.float)

# 创建数据对象
data = Data(edge_index=edge_index, edge_attr=edge_weight)

# 定义TransE模型
num_entities = 4  # 人和公司的总节点数
embedding_dim = 16  # 嵌入维度

# 使用 TransE 类进行嵌入
model = TransE(num_nodes=num_entities, hidden_channels=embedding_dim, num_relations=1)

# 训练模型
model.train()
optimizer = Adam(model.parameters(), lr=0.001)

# 创建头尾索引
head_index = edge_index[0]  # 投资人索引
tail_index = edge_index[1]   # 公司索引
relation_index = torch.zeros(head_index.size(0), dtype=torch.long)  # 所有关系的索引（假设只有一种关系）

for epoch in range(500):
    optimizer.zero_grad()
    # 使用头、尾索引、关系索引和边权重
    loss = model(head_index, relation_index, tail_index,)
    
    # 确保损失是标量
    loss = loss.mean()  # 取平均作为最终损失
    loss.backward()
    optimizer.step()
    
    if epoch % 50 == 0:
        print(f'Epoch {epoch}: Loss = {loss.item()}')

print("训练完成！")

# 获取嵌入结果
entity_embeddings = model.node_emb.weight.data
print("实体嵌入：")
print(entity_embeddings)


Epoch 0: Loss = -1.7937239408493042
Epoch 50: Loss = -2.4045767784118652
Epoch 100: Loss = -3.098721981048584
Epoch 150: Loss = -3.855877161026001
Epoch 200: Loss = -4.6569342613220215
Epoch 250: Loss = -5.423018932342529
Epoch 300: Loss = -6.173830509185791
Epoch 350: Loss = -6.916337013244629
Epoch 400: Loss = -7.653810501098633
Epoch 450: Loss = -8.387285232543945
训练完成！
实体嵌入：
tensor([[ 7.8829e-01, -4.4779e-01,  1.2912e+00, -1.2436e+00,  1.2583e+00,
          3.0577e-01, -1.8893e-01,  4.5175e-01, -1.8712e+00,  8.4413e-01,
          6.0943e-01, -6.8849e-01, -2.3973e-01, -4.1894e-01,  5.1160e-01,
         -1.8564e+00],
        [-1.0235e+00, -5.1289e-01,  2.0947e-01,  1.0953e+00,  1.3486e+00,
         -1.8944e-04,  3.2078e-05, -1.1329e+00, -3.7448e-01,  1.2809e+00,
          3.7390e-01,  6.2983e-01,  1.4020e+00,  6.1518e-01, -1.5975e+00,
         -1.6289e+00],
        [ 1.4022e+00, -3.8087e-01, -1.5629e+00, -1.1025e-04, -7.3842e-01,
          9.3512e-01,  7.4273e-01,  1.2148e+00,  3.404

In [26]:
model.forward?

Signature:
model.forward(
    head_index: torch.Tensor,
    rel_type: torch.Tensor,
    tail_index: torch.Tensor,
) -> torch.Tensor
Docstring:
Returns the score for the given triplet.

Args:
    head_index (torch.Tensor): The head indices.
    rel_type (torch.Tensor): The relation type.
    tail_index (torch.Tensor): The tail indices.
File:      ~/.local/lib/python3.10/site-packages/torch_geometric/nn/kge/transe.py
Type:      method

In [27]:
model?

Signature:      model(*args, **kwargs)
Type:           TransE
String form:    TransE(4, num_relations=1, hidden_channels=16)
File:           ~/.local/lib/python3.10/site-packages/torch_geometric/nn/kge/transe.py
Docstring:     
The TransE model from the `"Translating Embeddings for Modeling
Multi-Relational Data" <https://proceedings.neurips.cc/paper/2013/file/
1cecc7a77928ca8133fa24680a88d2f9-Paper.pdf>`_ paper.

:class:`TransE` models relations as a translation from head to tail
entities such that

.. math::
    \mathbf{e}_h + \mathbf{e}_r \approx \mathbf{e}_t,

resulting in the scoring function:

.. math::
    d(h, r, t) = - {\| \mathbf{e}_h + \mathbf{e}_r - \mathbf{e}_t \|}_p

.. note::

    For an example of using the :class:`TransE` model, see
    `examples/kge_fb15k_237.py
    <https://github.com/pyg-team/pytorch_geometric/blob/master/examples/
    kge_fb15k_237.py>`_.

Args:
    num_nodes (int): The number of nodes/entities in the graph.
    num_relations (int): The number of r